In [6]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import glob
import json

import pandas as pd

In [8]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Production Config':
                    d['prod'] = True if value == 'True' else False

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())


        big_data = big_data.append(d, ignore_index=True)

# files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
# i = 0
# for filename in files:
#     #print(f'parsing : {filename}')
#     json_file = filename.replace('backtest_result.txt','live_config.json')
#     with open(json_file) as f:
#         live_config = json.load(f)
#         config_type = live_config['config_name']
#
#     with open(filename, 'r') as file:
#         data = file.read().split('\n')
#         d = {'file': filename.replace('backtests/binance_production','')}
#         current_state = 'long'
#         for l in data:
#             l = l.split('|')
#             if len(l) == 4:
#                 key = l[1].strip()
#                 value = l[2].strip()
#
#                 if key == 'Passivbot Version':
#                     d['v'] = value
#                 if key == 'Symbol':
#                     d['s'] = value
#
#                 if key == 'Long':
#                     current_state = 'long'
#                     d['long'] = bool(value) or False
#                 if key == 'Short':
#                     current_state = 'short'
#                     d['short'] = bool(value) or False
#
#                 if key == 'Total gain' and current_state == 'long':
#                     d['tgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'long':
#                     d['adgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'long':
#                     d['bnkrl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'long':
#                     d['ebrl'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'long':
#                     d['maxsl'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'long':
#                     d['pamaxl'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'long':
#                     d['pastdl'] = float(value.strip())
#
#
#                 if key == 'Total gain' and current_state == 'short':
#                     d['tgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'short':
#                     d['adgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'short':
#                     d['bnkrs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'short':
#                     d['ebrs'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'short':
#                     d['maxss'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'short':
#                     d['pamaxs'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'short':
#                     d['pastds'] = float(value.strip())
#
#         d['prod'] = True
#         big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_271388/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_271388/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_271388/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_271388/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod
0,\1000SHIBUSDT\plots\2022-04-03T194816\backtest...,5.5,1000SHIBUSDT,True,52.53,0.1780,0.066776,0.644511,100.0,0.8613,12.442375,True,-96.020,-1.88000,0.146280,0.802360,5.447,0.10870,155.190833,NaN
1,\1000SHIBUSDT\plots\2022-04-03T204145\backtest...,5.5,1000SHIBUSDT,True,26.73,0.0850,0.070106,0.830417,100.0,0.9153,6.572917,True,-95.820,-1.88000,0.148225,0.803949,5.404,0.09669,155.191667,NaN
2,\1000SHIBUSDT\plots\2022-04-06T180851\backtest...,5.5,1000SHIBUSDT,True,90.63,0.2460,0.065637,0.629854,100.0,0.7673,12.442375,True,-91.820,-1.64000,0.124514,0.747041,5.959,0.08869,180.079167,True
3,\1000XECUSDT\plots\2022-04-06T155612\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
4,\1000XECUSDT\plots\2022-04-06T181135\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,\ZENUSDT\plots\2022-04-08T000448\backtest_resu...,5.5,ZENUSDT,True,78.47,0.2110,0.129356,0.981507,100.0,0.8117,14.342375,True,8.361,0.19300,0.132544,0.571631,19.640,0.25670,47.980000,False
282,\ZILUSDT\plots\2022-04-07T124653\backtest_resu...,5.5,ZILUSDT,True,11.86,0.0150,0.213304,1.171540,100.0,0.8399,35.809042,True,-90.810,-1.64000,0.128495,0.776683,5.828,0.07862,33.732625,True
283,\ZILUSDT\plots\2022-04-13T093427\backtest_resu...,5.5,ZILUSDT,True,24.10,0.0730,0.077813,0.534668,100.0,0.7921,13.360417,True,-90.650,-1.64000,0.125380,0.780698,5.950,0.07147,33.434708,False
284,\ZRXUSDT\plots\2022-04-07T125157\backtest_resu...,5.5,ZRXUSDT,True,57.37,0.1690,0.114260,0.708101,100.0,0.8835,22.423625,True,20.130,0.00183,0.141192,0.532109,93.390,0.63730,47.927917,True


Now filter the dataframe

In [9]:
# allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
#                    "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
#                    "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
#                    "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
#                    "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
#                    "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
#                    "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
#                    "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
#                    "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
#                    "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
#                    "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
#                    "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
#                    "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
#                    "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
#                    "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
#                    "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
#                    "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
#                    "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
#                    "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
#                    "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = [
    #"OMG","DUSK","GTC"
    # "LRC",
    # "--------------------------",
    # "SNX", "RSR",
    # "--------------------------",
    # "CELO", "ROSE", "OCEAN","SXP","DENT"
    # "SFP","ENS","DODO","AKRO","REEF","NEO",
    # "----------------------------------------",
    # "1000XEC","AKRO","ALPHA","ANKR","BAKE","BAND","BEL","BTS","C98","COMP",
    "-----------------------------------",
    # "DGB","DOGO","FLM","FLOW","HBAR",
    # "-----------------------------------",
    # "HOT","ICX","IOST","IOTA","KAVA","KLAY","LINA","LIT","MKR","MTL","NEO","NKN","OGN","ONT"
    #"QTUM","REEF","RLC","RVN","SC","SFP","STMX","TOMO","TRB","UNFI","WAVES", "ZIL", "ZRX"
    "COTI"
]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .85)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,...,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod,adgt_%
69,\COTIUSDT\plots\2022-04-06T223955\backtest_res...,5.5,COTIUSDT,True,-60.96,-7.900,0.159264,0.626131,5.975,-128100.0000,...,True,-91.86,-7.9000,0.216244,0.726058,17.830,0.19920,35.362500,True,-15.8000
70,\COTIUSDT\plots\2022-04-13T160021\backtest_res...,5.5,COTIUSDT,True,98.02,0.262,0.098702,0.509762,100.000,0.8332,...,True,-92.50,-1.6400,0.203694,0.718961,5.547,0.19750,222.012500,False,-1.3780
71,\COTIUSDT\plots\2022-04-13T160108\backtest_res...,5.5,COTIUSDT,True,-60.96,-7.900,0.159264,0.626131,5.975,-128100.0000,...,True,-91.86,-7.9000,0.216244,0.726058,17.830,0.19920,35.362500,False,-15.8000
72,\COTIUSDT\plots\2022-04-13T160216\backtest_res...,5.5,COTIUSDT,True,88.85,0.241,0.080641,0.716892,100.000,0.8171,...,True,-92.11,-1.6400,0.238432,0.757851,5.472,0.08211,222.012500,False,-1.3990
73,\COTIUSDT\plots\2022-04-13T160335\backtest_res...,5.5,COTIUSDT,True,62.36,0.181,0.147471,0.941815,100.000,0.8675,...,True,-92.69,-1.6400,0.239216,0.727650,5.206,0.18260,222.013750,False,-1.4590
74,\COTIUSDT\plots\2022-04-13T160415\backtest_res...,5.5,COTIUSDT,True,-62.65,-7.900,0.013203,0.065084,5.983,0.9198,...,True,-92.28,-7.9000,0.238226,0.754289,5.443,0.00000,35.872208,False,-15.8000
75,\COTIUSDT\plots\2022-04-13T160525\backtest_res...,5.5,COTIUSDT,True,65.55,0.188,0.157765,0.940861,100.000,0.8681,...,True,-22.52,-0.0342,0.098174,0.484426,64.510,0.57390,16.309708,False,0.1538
76,\COTIUSDT\plots\2022-04-13T160625\backtest_res...,5.5,COTIUSDT,True,-60.96,-7.900,0.159264,0.626131,5.975,-128100.0000,...,True,-91.86,-7.9000,0.216244,0.726058,17.830,0.19920,35.362500,False,-15.8000


In [10]:
df.drop_duplicates(
    subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file', 'prod', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_271388/2052413415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,prod,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
69,\COTIUSDT\plots\2022-04-06T223955\backtest_res...,True,COTIUSDT,-15.8000,-7.900,-7.9000,5.975,17.830,-128100.0000,0.19920,49.485417,0.626131,0.159264,35.362500,0.726058,0.216244
70,\COTIUSDT\plots\2022-04-13T160021\backtest_res...,False,COTIUSDT,-1.3780,0.262,-1.6400,100.000,5.547,0.8332,0.19750,36.001375,0.509762,0.098702,222.012500,0.718961,0.203694
71,\COTIUSDT\plots\2022-04-13T160108\backtest_res...,False,COTIUSDT,-15.8000,-7.900,-7.9000,5.975,17.830,-128100.0000,0.19920,49.485417,0.626131,0.159264,35.362500,0.726058,0.216244
72,\COTIUSDT\plots\2022-04-13T160216\backtest_res...,False,COTIUSDT,-1.3990,0.241,-1.6400,100.000,5.472,0.8171,0.08211,12.168750,0.716892,0.080641,222.012500,0.757851,0.238432
73,\COTIUSDT\plots\2022-04-13T160335\backtest_res...,False,COTIUSDT,-1.4590,0.181,-1.6400,100.000,5.206,0.8675,0.18260,26.338875,0.941815,0.147471,222.013750,0.727650,0.239216
74,\COTIUSDT\plots\2022-04-13T160415\backtest_res...,False,COTIUSDT,-15.8000,-7.900,-7.9000,5.983,5.443,0.9198,0.00000,49.870000,0.065084,0.013203,35.872208,0.754289,0.238226
75,\COTIUSDT\plots\2022-04-13T160525\backtest_res...,False,COTIUSDT,0.1538,0.188,-0.0342,100.000,64.510,0.8681,0.57390,27.318042,0.940861,0.157765,16.309708,0.484426,0.098174
